In [17]:
import pandas as pd
import requests 
import json

In [109]:
#538 Ratings
ratings = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv')
ratings = ratings[ratings.league=='Barclays Premier League']
ratings = ratings[['name','off','def','spi']].sort_values(by='name')
ratings['short_name'] = ['BOU','ARS','AVL','BRE','BHA','CHE','CRY','EVE','FUL','LEE','LEI','LIV','MCI','MUN','NEW','NFO','SOU','TOT','WHU','WOL']
ratings.drop(columns=['name'],inplace=True)

In [110]:
#Teams info - FPL
response_API = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
print(response_API.status_code)

data = response_API.text
json_obj = json.loads(data)
teams = pd.DataFrame.from_dict(json_obj['teams'])
teams = teams[['id','name','short_name']]

200


In [111]:
#Join ratings to teams data
teams = pd.merge(teams,ratings,how="left",on="short_name")

In [137]:
#Fixures  - FPL
response_API = requests.get('https://fantasy.premierleague.com/api/fixtures/')
print(response_API.status_code)

data = response_API.text
data_json=json.loads(data)
fixtures = pd.read_json(data)
fixtures = fixtures[['event','team_a','team_h']]

200


In [138]:
#Merge fixtures and teams info
teams_h = teams.add_suffix('_home')
teams_a = teams.add_suffix('_away')
fixtures = pd.merge(fixtures,teams_h,left_on="team_h",right_on="id_home").drop(columns=['team_h'])
fixtures = pd.merge(fixtures,teams_a,left_on="team_a",right_on="id_away").drop(columns=['team_a'])